In [1]:
import warnings
warnings.filterwarnings("ignore")

import os 
from pathlib import Path 
from langchain_ollama import OllamaEmbeddings
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np 

import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langchain_ollama import ChatOllama
from langchain_ollama.llms import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
import torch
import re
from typing import List
from langchain_core.runnables import chain
from langchain_core.runnables import RunnablePassthrough
import json


import gradio as gr
from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color="#76B900", bold=True)
pprint = partial(console.print, style=base_style)


from langserve import RemoteRunnable

## Load Dataset from CSV

In [2]:
data_dir = Path.cwd().parent / 'data/data_files'
file_name = 'prelim_cancer_data.csv'
data = pd.read_csv(data_dir / file_name)
data.head()

,Question,Answer
0,What is radiation therapy?,Radiation therapy (also called radiotherapy) i...
1,How is radiation therapy given?,Radiation therapy can be external beam or inte...
2,Who gets radiation therapy?,Many people with cancer need treatment with ra...
3,What does radiation therapy do to cancer cells?,"Given in high doses, radiation kills or slows ..."
4,How long does radiation therapy take to work?,Radiation therapy does not kill cancer cells r...


In [3]:
data.Question.tolist()[50]

'How does a Linac work?'

## Define Retrieval Utilities

In [4]:
def init_embedder(embedder_config, device):
    embedder_params = embedder_config['params']
    if embedder_config['backend'] == "HF":
        embedder = HuggingFaceEmbeddings(
            model_name=embedder_params['model_name'],
            model_kwargs={'device': device},
            encode_kwargs=embedder_params.get('encode_kwargs', {})
        )
        print(f"Embedder Initialized with {embedder_params['model_name']}")
    elif embedder_config['backend'] == "OLLAMA":
        embedder = OllamaEmbeddings(model=embedder_params['model_name'])
        print(f"Embedder Initialized with {embedder_params['model_name']}")
    else:
        raise NotImplementedError("Embedder backend not supported")
    return embedder


def init_vectorstore(embedder):
    # Create the FAISS index for storing embeddings
    embedding_size = len(embedder.embed_query("hello world"))  # Example to get embedding size
    index = faiss.IndexFlatL2(embedding_size)
    
    vector_store = FAISS(
        embedding_function=embedder,
        index=index,
        docstore=InMemoryDocstore(),
        index_to_docstore_id={}
    )
    return vector_store

def preprocess_text(text):
    """
    Preprocess the input text for embedding by normalizing and cleaning.
    :param text: input string to preprocess
    :return: cleaned and preprocessed string
    """
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespace
    return text

def create_documents_from_questions(questions, question_ids):
    for index in range(len(questions)):
        questions[index] = preprocess_text(questions[index])
    
    documents = []
    for text, doc_id in zip(questions, question_ids):
        _ = Document(
            page_content=text,
            metadata={"id" : doc_id}
        )
        documents.append(_)        
    return documents

## Create Retrieval Chain

In [5]:
# Define Configs and Settings For Retrieval
SCORE_THRESHOLD = 0.5
TOPK=3

EMBEDDER_CONFIG = {
    "backend" : "HF", 
    "params" : {
        "model_name" : "sentence-transformers/all-mpnet-base-v2",
        "encode_kwargs" : {'normalize_embeddings': False}
    }
}

device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [6]:
# Intialize Embedder and VectorStore
embedder = init_embedder(embedder_config=EMBEDDER_CONFIG, device=device)
vectorstore = init_vectorstore(embedder)

Embedder Initialized with sentence-transformers/all-mpnet-base-v2


In [7]:
# Add Documents to VectorStore
documents = create_documents_from_questions(data.Question.tolist(), data.index.tolist())
vectorstore.add_documents(documents=documents, ids=data.index.tolist())
print(f"{len(documents)} Documents added to Vector Store")

137 Documents added to Vector Store


In [8]:
# Mean Similarity Method. We can use other kind of averaging
def calculate_mean_similarity(sims):
    return np.mean(np.array(sims))

def calculate_max_similarity(sims):
    return np.max(np.array(sims))

# Define Custom Functions as Runnables to use in retrieval chain
@chain
def retriever(query: dict) -> List[Document]:
    """Custom Retriever Logic to filter based on SIM THRESHOLD"""
    query = preprocess_text(query.get('query',''))
    docs, scores = zip(*vectorstore.similarity_search_with_relevance_scores(query, k=TOPK))
    result = []
    for doc, score in zip(docs, scores):
        if score > SCORE_THRESHOLD:
            doc.metadata["score"] = score
            result.append(doc)
    return result

@chain
def format_retrieved_docs(documents: List[Document]) -> str:
    """Context Formatter"""
    docs = []
    for doc in documents:
        score = doc.metadata.get('score')
        doc_index = doc.metadata.get('id')
        answer = data.iloc[doc_index].Answer
        docs.append((doc.page_content, answer, score))

    context_text = "\n".join([f"Q: {ctx[0]}\nA: {ctx[1]}" for ctx in docs])
    #mean_sim = calculate_mean_similarity([ctx[2] for ctx in docs])
    max_sim = calculate_max_similarity([ctx[2] for ctx in docs])
    return {"context" : context_text, "retrieval_score" : max_sim}

In [8]:
ret_chain = retriever | format_retrieved_docs

## Create a Chain to fill User's InfoBase

In [11]:
from langchain.output_parsers import PydanticOutputParser
from langchain.schema.runnable.passthrough import RunnableAssign
from langchain_core.output_parsers import StrOutputParser


def RExtract(pydantic_class, llm, prompt):
    '''
    Runnable Extraction module
    Returns a knowledge dictionary populated by slot-filling extraction
    '''
    parser = PydanticOutputParser(pydantic_object=pydantic_class)
    instruct_merge = RunnableAssign({'format_instructions' : lambda x: parser.get_format_instructions()})
    def preparse(string):
        if '{' not in string: string = '{' + string
        if '}' not in string: string = string + '}'
        string = (string
            .replace("\\_", "_")
            .replace("\n", " ")
            .replace("\]", "]")
            .replace("\[", "[")
        )
        # print(string)  ## Good for diagnostics
        return string
    return instruct_merge | prompt | llm | preparse | parser

class KnowledgeBase(BaseModel):
    name: str = Field(default="unknown", description="Name of the user, first name and last name")
    age : str = Field(default="unknown", description="Age of the user")
    gender: str = Field(default="unknown", description="Gender of the user")
    query: str = Field(default="unknown", description="Detailed User's Query to answer. It should be framed in Question format.")
    summary: str = Field('unknown', description="Running summary of conversation. Update this with new input")
    response: str = Field('unknown', description="An ideal response to the user based on their new message")

parser_prompt = ChatPromptTemplate.from_template(
    "You are chatting with a user. The user just responded ('input'). Please update the knowledge base."
    "Record your response in the 'response' tag to continue the conversation."
    "Do not hallucinate any details, and make sure the knowledge base is not redundant."
    "Update the entries frequently to adapt to the conversation flow."
    "\n{format_instructions}"
    "\n\nOLD KNOWLEDGE BASE: {know_base}"
    "\n\nNEW MESSAGE: {input}"
    "\n\nNEW KNOWLEDGE BASE:"
)


llm = OllamaLLM(model='llama3.1:70b', temperature=0.4)
instruct_llm = llm #| StrOutputParser
extractor = RExtract(KnowledgeBase, instruct_llm, parser_prompt)
info_update = RunnableAssign({'know_base' : extractor})

## Initialize the knowledge base and see what you get
state = {'know_base' : KnowledgeBase()}
state['input'] = "What is radiation therapy and how can it be used to treat cancer?"
state = info_update.invoke(state)
print(state)

In [10]:
state['know_base'].query

'unknown'

In [12]:
@chain
def extract_query(state):
    return state.get('know_base').query

@chain
def merge_outputs(ret_results):
    return {**ret_results[0], **ret_results[1]}

internal_chain = (
    info_update
    | RunnableAssign({"query" : extract_query})
    | (lambda x: (x, ret_chain.invoke(x)))
    | merge_outputs
)

In [6]:
internal_chain = RemoteRunnable("http://localhost:8000/retrieve")

In [7]:
state = {'know_base' : dict(KnowledgeBase())}
state['input'] = "What is radiation therapy and how can it be used to treat cancer?"
state = internal_chain.invoke(state)

In [9]:
state = json.loads(state)

In [10]:
state.keys()

dict_keys(['know_base', 'input', 'query', 'context', 'retrieval_score'])

In [41]:
# Implementing LLM as Judge to rate retrieved context according to given query
# Data model
class GradeDocuments(BaseModel):
    """Score for relevance check on retrieved documents."""

    relevancy_score: float = Field(
        description="Documents are relevant to the question, 0 to 1."
    )

# Prompt
system = """You are a grader assessing relevance of a retrieved context to a user question. \n 
    If the context contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a score between 0 and 1 to indicate whether the context is relevant to question.\n
    If the context is empty, then assign 0 relevancy.\n
    0 means 0 % relevancy and 1 means 100 % relevancy. The score can be anything in the range of 0 and 1.\n
    Use the following scale to assign a score between 0 and 1: \n
    - **1**: The context is highly relevant, directly answering or providing strong support for the question.\n
    - **0**: The context is entirely irrelevant, containing no useful information related to the question\n
    - **0.1 - 0.9**: Partial relevance, where a score closer to **1** means more useful and related context, and a score closer to **0** means less related or only marginally helpful.\n
    {format_instructions}
    """

grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved context: \n\n {context} \n\n User question: {query}"),
    ]
)

# LLM with function call
grader_llm = OllamaLLM(model='llama3.1:8b', temperature=0)
context_grader = RExtract(GradeDocuments, grader_llm, grade_prompt)

In [24]:
internal_chain = RemoteRunnable("http://localhost:8000/retrieve")
context_grader = RemoteRunnable("http://localhost:8000/grade_context")
external_chain = RemoteRunnable("http://localhost:8000/chat")

In [28]:
#state = {'know_base' : KnowledgeBase()}
state = {'know_base' : dict(KnowledgeBase())}
state['input'] = "What is the time span for radiation therapy to show effects?"
#state['input'] = "What is the state of feeling during radiation therapy?"
state = json.loads(internal_chain.invoke(state))
response = context_grader.invoke(state)

In [35]:
state['summary'] = state['know_base']['summary']
state['output'] = ""

In [36]:
external_chain.invoke(state)

"The effects of radiation therapy can vary depending on several factors, such as the type and stage of cancer, the location of the tumor, and the individual's overall health.\n\n Generally, patients may start noticing improvements in their symptoms within 2-4 weeks after starting radiation therapy. However, it's essential to note that some people may not experience significant changes until a few months later, while others might see improvements sooner.\n\nWould you like me to provide more information on what to expect during and after radiation therapy?"

In [14]:
response

In [47]:
state['retrieval_score']

0.6203085458796949

In [40]:
state['output'] = ""

In [49]:
external_prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "You are a chatbot for NU Medicine, and you are providing users cancer treatment assistance."
        " Please chat with them! Stay concise and clear!"
        " Your running knowledge base is: {know_base}."
        " This is for you only; Do not mention it!"
        " \nUsing that, we retrieved the following: {context}\n"
        "\nHere is the query that user want to get answered: {query}\n"
        "\nKeep asking follow up questions, until user provides basic info like name, age and gender\n"
        "\nModerate your tone according to user's age and gender\n"
        "If they provide info and the retrieval fails, ask to confirm their name, age, gender and query"
        " Do not ask them any other personal info."
    )),
    ("assistant", "{output}"),
    ("user", "{input}"),
])

external_chain = (
                external_prompt 
                | OllamaLLM(model='llama3.1:70b', temperature=0.4)
                )

In [58]:
external_chain.invoke(state)

"Hello! I'd be happy to help you with your question about radiation therapy.\n\nRadiation therapy, also known as radiotherapy, is a treatment that uses high-energy particles or waves, such as X-rays, gamma rays, electron beams, or protons, to destroy or damage cancer cells. It can be used to treat various types of cancer, including breast, lung, prostate, and brain cancers, among others.\n\nBefore I provide more information, could you please tell me a bit about yourself? What's your name, age, and gender? This will help me tailor my response to your specific needs and concerns."

In [56]:
state = {'know_base' : KnowledgeBase()}

def chat_gen(message, history=[], return_buffer=True):

    ## Pulling in, updating, and printing the state
    global state
    state['input'] = message
    state['history'] = history
    state['output'] = "" if not history else history[-1][1]

    ## Generating the new state from the internal chain
    state = internal_chain.invoke(state)
    #print("State after chain run:")
    #pprint({k:v for k,v in state.items() if k != "history"})
    
    ## Streaming the results
    buffer = ""
    is_first_token = False
    retrieval_score = state.get('retrieval_score')
    relevancy_socre = context_grader.invoke(state).relevancy_score
    for token in external_chain.stream(state):
        if not is_first_token:
            is_first_token = True
            token = "RETRIEVAL SIM : {:.3f}\n RELEVANCY_SCORE : {:.3f}\n".format(retrieval_score, relevancy_socre) + token
        buffer += token
        yield buffer if return_buffer else token
    #yield "RETRIEVAL SIM : {:.3f}\n RELEVANCY_SCORE : {:.3f}".format(retrieval_score, relevancy_socre)

    

def queue_fake_streaming_gradio(chat_stream, history = [], max_questions=8):

    ## Mimic of the gradio initialization routine, where a set of starter messages can be printed off
    for human_msg, agent_msg in history:
        if human_msg: print("\n[ Human ]:", human_msg)
        if agent_msg: print("\n[ Agent ]:", agent_msg)

    ## Mimic of the gradio loop with an initial message from the agent.
    for _ in range(max_questions):
        message = input("\n[ Human ]: ")
        print("\n[ Agent ]: ")
        history_entry = [message, ""]
        for token in chat_stream(message, history, return_buffer=False):
            print(token, end='')
            history_entry[1] += token
        history += [history_entry]
        print("\n")

## history is of format [[User response 0, Bot response 0], ...]
chat_history = [[None, "Hello! I'm your NU Medicine Agent! How can I help you?"]]

## Simulating the queueing of a streaming gradio interface, using python input
# queue_fake_streaming_gradio(
#     chat_stream = chat_gen,
#     history = chat_history
# )

In [8]:
## Test 
grader_input = {'context': 'Q: what are the possible side effects risks and complications of radiation therapy in lung cancer\nA: overall radiotherapy is a local treatment any side effects usually happen in the parts of the body where radiation passes through radiotherapy usually does not damage normal organs outside the path of the treatment beam most patients tire easily during their course of radiation treatment this fatigue occurs two to three weeks after treatment starts and gradually increases as treatment continues it may become severe and seriously limit your ability to engage in normal daily activities typically fatigue lessens one to two months after radiotherapy is completed if you experience fatigue it is important to get enough rest however at the same time your doctor may suggest that you stay as active as possible some patients experience hair loss on the chest included in the radiation field depending on how much radiation is delivered this may be temporary or permanent skin irritation is common after a few weeks of radiation therapy the affected area may be red dry tender and itchy this reaction can become quite severe during a long course of treatment it helps to keep the skin clean with gentle soap and warm water to dry it well and to avoid very hot water while bathing sunscreen should be used when skin in the treated area is exposed to sunlight do not use perfumes cosmetics or deodorants in the treatment area after daily treatments you may apply an unscented cream or lotion temporary loss of appetite is a possibility esophagitis or inflammation of the esophagus the tube that carries food from the mouth to the stomach is common if radiotherapy is directed at the lymph nodes in the central chest or if the tumor is near the esophagus this condition can be severe the esophagus is very sensitive to radiation and the symptoms are worse in patients who also receive chemotherapy esophagitis can make it difficult to swallow and some patients need intravenous iv pain medication or fluids to get through the full course of radiation treatment some people experience esophagitis as the sensation of a lump or discomfort before it becomes overtly painful some foods such as spicy or acidic foods or bread feel worse than others it is important to remember that although certain foods may cause pain with swallowing they will not cause damage avoid foods that cause pain until about one month after the final day of radiation the inflammation generally plateaus toward the end of treatment and starts to resolve within two to three weeks after treatment in most patients an inflammation of the lungs called radiation pneumonitis may develop three to six months after radiotherapy is over it causes coughing and shortness of breath as well as fever in most cases it calls for no specific treatment and gets better within two to four weeks it can occasionally happen earlier and require steroids andor oxygen decrease in a specific type of white blood cell called lymphocytes is another side effect of radiotherapy\nQ: what does radiation therapy do to cancer cells\nA: given in high doses radiation kills or slows the growth of cancer cells radiation therapy is used to 1 treat cancer radiation can be used to cure cancer to prevent it from returning or to stop or slow its growth 2 reduce symptoms when a cure is not possible radiation may be used to treat pain and other problems caused by the cancer tumor or it can prevent problems that may be caused by a growing tumor such as blindness or loss of bowel and bladder control\nQ: how effective is modern radiation treatment of lung cancer\nA: it is extremely important to remember that inoperable does not mean incurable when it comes to lung cancer in fact an increasing number of patients are being treated with a nonsurgical approach across all stages of this disease treatment effectiveness depends on the stage of disease in early stage inoperable disease that is treated with radiotherapy alone control of the local disease is typical in more advanced disease a combination of chemotherapy and radiation is delivered with curative intent cure rates are lower but still possible with disease spread to the lymph nodes within the chest the medical or radiation oncologist may propose a combination of chemotherapy and radiation therapy for a patient who is otherwise well when cure is not a possibility doctors recommend palliative treatment this is the use of medications chemotherapy radiation therapy or other measures to relieve symptoms of lung cancer without eliminating the tumor the doctor will use smaller doses of radiation therapy to avoid side effects at some point if you and your oncologist or primary care physician agree that active treatment is no longer advisable hospice care can provide comfort and support pain relief is a very important part of treating lung cancer although many effective treatments are available and there are devices for delivering medication on demand without overdosing many cancer patients still do not receive adequate pain relief when the needs of the patient are clearly expressed doctors can better provide appropriate care\nQ: what are possible side effects risks and complications of radiation therapy in breast cancer\nA: there are usually no immediate side effects from each radiation treatment given to the breast patients do not develop nausea or hair loss on the head from radiation therapy to the breast most patients develop mild fatigue that builds up gradually over the course of therapy this slowly goes away one to two months following the radiation therapy most patients develop dull aches or sharp shooting pains in the breast that may last for a few seconds or minutes it is rare for patients to need any medication for this the most common side effect needing attention is skin reaction most patients develop reddening dryness and itching of the skin after a few weeks some patients develop substantial irritation skin care recommendations include keeping the skin clean using gentle soap and warm but not hot water avoiding extreme temperatures while bathing avoiding trauma to the skin and sun exposure use clothing as a barrier or a sunscreen with at least spf 30 avoiding shaving the treatment area with a razor blade use an electric razor if necessary avoiding use of perfumes cosmetics aftershave or deodorants in the treatment area use cornstarch with or without baking soda in place of deodorants using only recommended unscented creams or lotions after daily treatment some patients develop a sunburnlike reaction with blistering and peeling of the skin called moist desquamation this usually occurs in the fold under the breast or in the fold between the breast and the arm or sometimes in the area given a radiation boost most people with a limited area of moist desquamation can continue treatment without interruption sometimes topical medications those applied to the surface of the body are prescribed when treatment must be interrupted the skin usually heals enough to allow radiation to be resumed in five to seven days skin reactions usually heal completely within a few weeks of completing radiotherapy minor complications include slight swelling of the breast during radiotherapy this usually goes away within six to 12 months the skin becomes darker during the course of radiotherapy similar to tanning from the sun in most cases this also fades gradually over six to 12 months most women will have aches or pains from time to time in the treated breast or the muscles surrounding the breast even years after treatment the reason why this happens is not clear however these pains are harmless although annoying they are not usually a sign that the cancer is reappearing rarely patients may develop a rib fracture years following treatment this occurs in less than one percent of patients treated by modern approaches these heal slowly by themselves more serious complications include radiation therapy given to the axillary lymph nodes can increase the risk of patients developing arm swelling lymphedema following axillary armpit dissection radiation therapy given to the axillary lymph nodes may rarely cause numbness tingling or even pain and loss of strength in the hand and arm years after treatment some patients develop radiation pneumonitis a lung reaction that causes a cough shortness of breath and fevers three to nine months after completing treatment fortunately it is usually mild enough that no specific treatment is needed and it goes away within two to four weeks with no longterm complications radiotherapy may damage the heart fortunately modern radiation techniques treat much less of the heart than treatments used in the past current studies have found no increased risk of serious heart disease in patients treated with modern techniques even 10 to 20 years after radiotherapy treatment was given however there is still some uncertainty about the risks of radiation causing heart disease for individuals who smoke or have preexisting heart disease or for those who receive certain chemotherapy drugs it is likely that such risks will also be found to be very small fortunately newer radiation techniques treat patients in the prone position patient lying on their stomach pulling the breast away from the heart or use deep inspiration technique taking a deep breath and pulling the chest away from the heart during radiation therapy both techniques may further reduce radiation related heart injury women age 45 or younger at the time of treatment may have a slightly increased risk by a few percent at most of developing cancer of the other breast with time compared with the risk they would have if they did not undergo radiation there is a very small risk perhaps one in 1000 individuals that cancers may develop five 10 20 or more years later in the skin muscle bone or lung directly in the area of treatment patients may rarely develop a breakdown of the skin or such severe pain in the breast that surgery is needed for treatment\nQ: is radiation therapy used with other types of cancer treatment\nA: yes radiation therapy is often used with other cancer treatments here are some examples 1 radiation therapy and surgery radiation may be given before during or after surgery doctors may use radiation to shrink the size of the cancer before surgery or they may use radiation after surgery to kill any cancer cells that remain sometimes radiation therapy is given during surgery so that it goes straight to the cancer without passing through the skin radiation therapy given during surgery is called intraoperative radiation 2 radiation therapy chemotherapy and immunotherapy radiation may be given before during or after chemotherapy and immunotherapy before or during these other treatments radiation therapy can shrink the cancer so that the treatments work better after treatment radiation can be used to kill any cancer cells that remain\nQ: what are possible side effects risks and complications of radiation therapy in lymphoma cancer\nA: while side effects vary from person to person some are typical radiation treatments usually have no immediate side effects most patients have mild fatigue that builds up gradually over the course of therapy this slowly goes away one to two months after radiation therapy some patients have some reddening dryness and itchy skin after a few weeks skin reactions usually heal completely within a few weeks of completing treatment if you have radiation to the abdomen or pelvis diarrhea frequent bowel movements or appetite loss may occur medication can ease these side effects generally side effects gradually stop once treatment is complete\nQ: what are possible side effects risks and complications of radiation therapy in brain cancer\nA: the side effects of radiation therapy to the brain may not occur until two to three weeks after the start of your therapy many people experience hair loss but the amount varies from person to person hair may grow back once therapy is finished skin irritation is the second most frequently reported side effect the skin around your ears and scalp may become dry itchy red or tender do not try to treat this side effect on your own seek professionalism medical treatment as soon as it occurs fatigue is another possible side effect the best way to fight fatigue is to get on a daily exercise regimen that is tolerable and sustainable eat a healthy diet and rely on friends and family for support your normal energy levels should return about six weeks after you finish your therapy fatigue may be the worst two to three weeks after you complete prolonged multiweek radiation treatment edema swelling of the brain is also a common side effect tell your oncologist if you have a headache or a feeling of pressure the doctor may prescribe medications to help reduce brain swelling prevent seizures or control pain if you receive chemotherapy and radiation therapy at the same time you may experience more severe side effects your doctor can suggest ways to ease these symptoms other possible side effects include hearing problems nausea vomiting loss of appetite memory or speech problems headaches radiation is a powerful weapon against cancer cells but sometimes it kills healthy brain tissue as well this severe side effect is called radiation necrosis necrosis a late effect of high doses of radiation can cause headaches seizures or even death in a small number of cases this can occur six months to a few years after treatment however there is less risk of necrosis today because of newer targeted radiation therapies and the emergence of powerful imaging brain mapping and information technologies other complications include neurologic deficits this usually depends on the area of the brain being treated cognitive problems seizures headaches return of tumor growth in children radiation may damage the pituitary gland and other parts of the brain this could cause learning problems or slow growth and development additionally radiation during childhood increases the risk of developing tumors later in life researchers are studying chemotherapy as an alternative to radiation therapy in children with brain tumors\nQ: what does radiation therapy do to healthy cells\nA: radiation not only kills or slows the growth of cancer cells it can also affect nearby healthy cells the healthy cells almost always recover after treatment is over but sometimes people may have side effects that are severe or do not get better other side effects may show up months or years after radiation therapy is over these are called late side effects doctors try to protect healthy cells during treatment by 1 using as low a dose of radiation as possible the radiation dose is balanced between being high enough to kill cancer cells yet low enough to limit damage to healthy cells 2 spreading out treatment over time you may get radiation therapy once a day or in smaller doses twice a day for several weeks spreading out the radiation dose allows normal cells to recover while cancer cells die 3 aiming radiation at a precise part of your body some types of radiation therapy allow your doctor to aim high doses of radiation at your cancer while reducing radiation to nearby healthy tissue these techniques use a computer to deliver precise radiation doses to a cancer tumor or to specific areas within the tumor\nQ: what are possible side effects risks and complications of radiation therapy in prostate cancer\nA: initially most patients do not have side effects radiation side effects build with time they tend to be worse toward the end of a week and better after the weekend break as radiation treatment goes on it may cause you to feel tired while adequate rest is important doctors will usually advise you to stay as active as you can you may have some rectal irritation such as bowel urgency discomfort diarrhea andor frequent and uncomfortable urination also skin in the treated area may become dry redness tenderness or hair loss is unusual radiation therapy can cause impotence in some men at a similar rate as nervesparing prostatectomy typically radiation induced impotence will respond to medications to improve erectile function there is also a small risk of late bladder or rectal problems with radiation these may appear one to three years after radiation therapy your radiation oncologist will monitor you for late complications and check your psa in your followup appointments\nQ: what are the possible side effects risks and complications of radiation therapy in colorectal cancer cancer\nA: side effects that develop during treatment vary from person to person however some side effects are typical there are usually no immediate side effects from each radiation treatment most patients gradually develop mild fatigue over the course of therapy this slowly goes away one to two months following treatment some patients develop some redness dryness and itching of the skin after a few weeks other patients develop substantial irritation skin reactions usually heal completely within a few weeks of completing treatment diarrhea frequent bowel movements or appetite loss is likely to occur at some point during the course of treatment generally side effects stop gradually once treatment ceases but bowel function may remain different from what it was before the disease started', 'query': 'What are the benefits and drawbacks of radiotherapy in cancer treatment?'}
# grader_input = {
#     "context" : "",
#     "query" : "What are the benefits and drawbacks of radiotherapy in cancer treatment?"

# }
print(grader_input.keys())


class GradeDocuments(BaseModel):
    """Score for relevance check on retrieved documents."""
    relevancy_score: float = Field(
        description="Documents are relevant to the question, 0 to 1."
    )

# Prompt
system = """You are a grader assessing relevance of a retrieved context to a user question. \n 
    If the context contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a score between 0 and 1 to indicate whether the context is relevant to question.\n
    If the context is empty, then assign 0 relevancy.\n
    0 means 0 % relevancy and 1 means 100 % relevancy. The score can be anything in the range of 0 and 1.\n
    Use the following scale to assign a score between 0 and 1: \n
    - **1**: The context is highly relevant, directly answering or providing strong support for the question.\n
    - **0**: The context is entirely irrelevant, containing no useful information related to the question\n
    - **0.1 - 0.9**: Partial relevance, where a score closer to **1** means more useful and related context, and a score closer to **0** means less related or only marginally helpful.\n
    """

grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved context: \n\n {context} \n\n User question: {query}"),
    ]
)

# LLM with function call
grader_llm = ChatOllama(model='llama3.1:8b', temperature=0.4).with_structured_output(GradeDocuments)
context_grader = grade_prompt | grader_llm

dict_keys(['context', 'query'])


In [9]:
#print(context_grader.invoke(grader_input).relevancy_score)

AttributeError: 'NoneType' object has no attribute 'relevancy_score'

In [57]:
state = {'know_base' : KnowledgeBase()}

chatbot = gr.Chatbot(value=[[None, "Hello! I'm your NU Medicine Agent! How can I help you?"]])
demo = gr.ChatInterface(chat_gen, chatbot=chatbot).queue().launch(debug=True, share=True)

Running on local URL:  http://127.0.0.1:7860


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Running on public URL: https://9c21802d8cf5463b1d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://9c21802d8cf5463b1d.gradio.live
